In [103]:
from skmultilearn.ext import Keras
from keras.models import Sequential
from keras.layers import Dense
import numpy
import sklearn.metrics as metrics
from skmultilearn.dataset import load_dataset

In [ ]:
X_train, y_train, feature_names, label_names = load_dataset('emotions', 'train')
X_test, y_test, _, _ = load_dataset('emotions', 'test')

# Single-class Keras classifier

We train a two-layer neural network using Keras and tensortflow as backend (feel free to use others), the network is fairly simple 12 x 8 RELU that finish with a sigmoid activator optimized via binary cross entropy. This is a case from the [Keras example page](https://keras.io/scikit-learn-api/). Note that the model creation function must create a model that accepts an input dimension and outpus a relevant output dimension. The Keras wrapper from scikit-multilearn will pass relevant dimensions upon fitting. 

In [105]:
def create_model_single_class(input_dim, output_dim):
	# create model
	model = Sequential()
	model.add(Dense(12, input_dim=input_dim, activation='relu'))
	model.add(Dense(8, activation='relu'))
	model.add(Dense(output_dim, activation='sigmoid'))
	# Compile model
	model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])
	return model

Let's use it with a problem transformation method which converts multi-label classification problems to single-label single-class problems, ex. Binary Relevance which trains a classifier per label.

In [123]:
from skmultilearn.problem_transform import BinaryRelevance

In [126]:
clf = BinaryRelevance(classifier=Keras(create_model_single_class), require_dense=[True,True])

In [127]:
clf.fit(X_train,y_train)

BinaryRelevance(classifier=<__main__.Keras object at 0x7f4d0b040978>,
        require_dense=[True, True])

In [130]:
y_pred = clf.predict(X_test)

In [133]:
metrics.hamming_loss(y_test, y_pred)

0.25495049504950495

# Single-class Keras classifier

We now train a multi-class neural network using Keras and tensortflow as backend (feel free to use others) optimized via categorical cross entropy. This is a case from the [Keras multi-class tutorial](https://machinelearningmastery.com/multi-class-classification-tutorial-keras-deep-learning-library/). Note again that the model creation function must create a model that accepts an input dimension and outpus a relevant output dimension. The Keras wrapper from scikit-multilearn will pass relevant dimensions upon fitting. We must also tel the Keras wrapper that this is a multi-class case. We use the Label Powerset multi-label to multi-class transformation approach, but this can also be used with all the advanced label space division methods available in scikit-multilearn.

In [136]:
from skmultilearn.problem_transform import LabelPowerset

In [137]:
def create_model_multiclass(input_dim, output_dim):
	# create model
	model = Sequential()
	model.add(Dense(8, input_dim=input_dim, activation='relu'))
	model.add(Dense(output_dim, activation='softmax'))
	# Compile model
	model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
	return model

In [138]:
clf = LabelPowerset(classifier=Keras(create_model_multiclass, multi_class=True), require_dense=[True,True])

In [139]:
clf.fit(X_train,y_train)

LabelPowerset(classifier=<__main__.Keras object at 0x7f4d09229be0>,
       require_dense=[True, True])

In [140]:
y_pred = clf.predict(X_test)

In [142]:
metrics.accuracy_score(y_test, y_pred)

0.22277227722772278